# 01 Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os

# 02 Importing Data

In [6]:
#creating path
path=r'/Users/serenakunzmann/Documents/CareerFoundry-Python/03-2025 Instacart Basket Analysis/Data'

ords_prods_merge=pd.read_pickle(os.path.join(path,'Prepared Data','Data Consistency','ords_prods_merge_group_agg.pkl'))

In [10]:
ords_prods_merge.shape

(32404859, 21)

In [12]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,77,7,9.0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,91,16,12.5,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,23,19,4.4,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,23,19,4.7,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,54,17,1.0,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer


# 03 Grouping Data & Aggregating Variables

In [14]:
ords_prods_merge['max_order']=ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_48396/2690353080.py:1: FutureWarning: The provided callable <function max at 0x10613a5c0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order']=ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [16]:
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,77,7,9.0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,91,16,12.5,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,23,19,4.4,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,23,19,4.7,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,54,17,1.0,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer


In [18]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [20]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [22]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [24]:
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

# 04 Task

In [ ]:
#The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [28]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean','median','min','max','sum','count']})

prices                                         
                      mean median  min   max          sum     count
loyalty_flag                                                       
Loyal customer    7.773487    7.4  1.0  25.0   79943268.2  10284093
New customer      7.801077    7.4  1.0  25.0   48709847.1   6243990
Regular customer  7.798145    7.4  1.0  25.0  123809407.0  15876776

observations: prices of products purchased by loyal customers don't really differ compared to new and regular customers when looking at the average price. However, total prices of products purchase differ due to the # of customers that fall into each group. 

The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders

In [38]:
#creating avg_price column by finding the mean price grouped by user_id
ords_prods_merge['avg_price']=ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_48396/2830821007.py:1: FutureWarning: The provided callable <function mean at 0x10613afc0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price']=ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [40]:
#checking avg_price has been added successfully
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,0,2539329,1,1,2,8,NaN,196,1,0,...,7,9.0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,16,12.5,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,19,4.4,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,19,4.7,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,17,1.0,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797


In [42]:
#creating spending flag column
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [44]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [46]:
#checking new spending_flag column successfully labeled customer buckets
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,9.0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,12.5,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,4.4,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,4.7,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,1.0,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender


In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column.

In [50]:
#creating median_order column by finding median of days_since_prior_order grouped by user id
ords_prods_merge['median_order']=ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_48396/1875046042.py:2: FutureWarning: The provided callable <function median at 0x106281c60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_order']=ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [52]:
#checking median_order column was added successfully
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_order
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,Low-range product,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5


In [54]:
#creating order frequency flag column
ords_prods_merge.loc[ords_prods_merge['median_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [58]:
ords_prods_merge.loc[(ords_prods_merge['median_order'] >10) & (ords_prods_merge['median_order'] <=20), 'order_frequency_flag'] = 'Regular customer'

In [60]:
ords_prods_merge.loc[ords_prods_merge['median_order'] <=10, 'order_frequency_flag'] = 'Frequent customer'

In [62]:
#checking order freqency flag was added successfully
ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_order,order_frequency_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,1,2539329,1,1,2,8,NaN,14084,2,0,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2,2539329,1,1,2,8,NaN,12427,3,0,...,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,3,2539329,1,1,2,8,NaN,26088,4,0,...,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,4,2539329,1,1,2,8,NaN,26405,5,0,...,Low-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [72]:
#exporting data as pickle
ords_prods_merge.to_pickle(os.path.join(path,'Prepared Data','Data Consistency','order_prods_merge_grouped_agg.pkl'))